In [1]:
# this converts a zst file to csv
#
# it's important to note that the resulting file will likely be quite large
# and you probably won't be able to open it in excel or another csv reader
#
# arguments are inputfile, outputfile, fields
# call this like
# python to_csv.py wallstreetbets_submissions.zst wallstreetbets_submissions.csv author,selftext,title

import zstandard
import os
import json
import sys
import csv
from datetime import datetime
import logging.handlers


# put the path to the input file
input_file_path = "/home/jesse/Downloads/dogecoin_comments.zst"
# put the path to the output file, with the csv extension
output_file_path = "/home/jesse/Downloads/dogecoin_comments.csv"
# if you want a custom set of fields, put them in the following list. If you leave it empty the script will use a default set of fields
fields = []

log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


def read_and_decode(reader, chunk_size, max_window_size, previous_chunk=None, bytes_read=0):
	chunk = reader.read(chunk_size)
	bytes_read += chunk_size
	if previous_chunk is not None:
		chunk = previous_chunk + chunk
	try:
		return chunk.decode()
	except UnicodeDecodeError:
		if bytes_read > max_window_size:
			raise UnicodeError(f"Unable to decode frame after reading {bytes_read:,} bytes")
		return read_and_decode(reader, chunk_size, max_window_size, chunk, bytes_read)


def read_lines_zst(file_name):
	with open(file_name, 'rb') as file_handle:
		buffer = ''
		reader = zstandard.ZstdDecompressor(max_window_size=2**31).stream_reader(file_handle)
		while True:
			chunk = read_and_decode(reader, 2**27, (2**29) * 2)
			if not chunk:
				break
			lines = (buffer + chunk).split("\n")

			for line in lines[:-1]:
				yield line, file_handle.tell()

			buffer = lines[-1]
		reader.close()


if __name__ == "__main__":
	if len(sys.argv) >= 3:
		input_file_path = sys.argv[1]
		output_file_path = sys.argv[2]
		fields = sys.argv[3].split(",")

	is_submission = "submission" in input_file_path
	if not len(fields):
		if is_submission:
			fields = ["author","title","score","created","link","text","url"]
		else:
			fields = ["author","score","created","link","body"]

	file_size = os.stat(input_file_path).st_size
	file_lines, bad_lines = 0, 0
	line, created = None, None
	output_file = open(output_file_path, "w", encoding='utf-8', newline="")
	writer = csv.writer(output_file)
	writer.writerow(fields)
	try:
		for line, file_bytes_processed in read_lines_zst(input_file_path):
			try:
				obj = json.loads(line)
				output_obj = []
				for field in fields:
					if field == "created":
						value = datetime.fromtimestamp(int(obj['created_utc'])).strftime("%Y-%m-%d %H:%M")
					elif field == "link":
						if 'permalink' in obj:
							value = f"https://www.reddit.com{obj['permalink']}"
						else:
							value = f"https://www.reddit.com/r/{obj['subreddit']}/comments/{obj['link_id'][3:]}/_/{obj['id']}/"
					elif field == "author":
						value = f"u/{obj['author']}"
					elif field == "text":
						if 'selftext' in obj:
							value = obj['selftext']
						else:
							value = ""
					else:
						value = obj[field]

					output_obj.append(str(value).encode("utf-8", errors='replace').decode())
				writer.writerow(output_obj)

				created = datetime.utcfromtimestamp(int(obj['created_utc']))
			except json.JSONDecodeError as err:
				bad_lines += 1
			file_lines += 1
			if file_lines % 100000 == 0:
				log.info(f"{created.strftime('%Y-%m-%d %H:%M:%S')} : {file_lines:,} : {bad_lines:,} : {(file_bytes_processed / file_size) * 100:.0f}%")
	except KeyError as err:
		log.info(f"Object has no key: {err}")
		log.info(line)
	except Exception as err:
		log.info(err)
		log.info(line)

	output_file.close()
	log.info(f"Complete : {file_lines:,} : {bad_lines:,}")

/tmp/ipykernel_14566/2987543011.py:106: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  created = datetime.utcfromtimestamp(int(obj['created_utc']))
2013-12-24 21:42:14 : 100,000 : 0 : 2%
2014-01-02 21:48:00 : 200,000 : 0 : 2%
2014-01-15 20:47:25 : 300,000 : 0 : 4%
2014-01-22 07:34:34 : 400,000 : 0 : 4%
2014-01-26 09:21:42 : 500,000 : 0 : 6%
2014-01-30 12:39:30 : 600,000 : 0 : 6%
2014-02-03 16:07:22 : 700,000 : 0 : 6%
2014-02-07 23:20:19 : 800,000 : 0 : 8%
2014-02-12 15:19:49 : 900,000 : 0 : 8%
2014-02-16 00:46:17 : 1,000,000 : 0 : 11%
2014-02-20 18:45:45 : 1,100,000 : 0 : 11%
2014-02-27 17:04:09 : 1,200,000 : 0 : 11%
2014-03-07 14:47:27 : 1,300,000 : 0 : 13%
2014-03-17 06:12:07 : 1,400,000 : 0 : 13%
2014-03-26 22:48:01 : 1,500,000 : 0 : 15%
2014-04-04 03:02:49 : 1,600,000 : 0 : 15%
2014-04-10 16:22:2

In [3]:
import pandas as pd

# 原始 CSV 路径
input_path = "/home/jesse/Projects/myprojs/MT_1/01_Data_Collection/Dogecoin_CSV/dogecoin_comments.csv"

# 目标输出路径
output_path = "/home/jesse/Projects/myprojs/MT_1/01_Data_Collection/Dogecoin_CSV/Dogecoin_Reddit_2021-01-01_to_2022-12-31.csv"

# 读取 CSV 文件
df = pd.read_csv(input_path)

# 将 created 字段转换为 datetime 类型
df['created'] = pd.to_datetime(df['created'], errors='coerce')

# 筛选日期范围
start_date = pd.Timestamp('2021-01-01')
end_date = pd.Timestamp('2022-12-31 23:59:59')
filtered_df = df[(df['created'] >= start_date) & (df['created'] <= end_date)]

# 保存过滤后的数据
filtered_df.to_csv(output_path, index=False)
print(f"✅ 已保存筛选后的数据到：{output_path}")
print(f"共保留评论数：{len(filtered_df)}")


✅ 已保存筛选后的数据到：/home/jesse/Projects/myprojs/MT_1/01_Data_Collection/Dogecoin_CSV/Dogecoin_Reddit_2021-01-01_to_2022-12-31.csv
共保留评论数：8134067
